### Problem 1

__We will use a full day worth of tweets as an input (there are total of 4.4M tweets in this file, but you only need to read 1M):__ http://rasinsrv07.cstcis.cti.depaul.edu/CSC455/OneDayOfTweets.txt

__a. Create a 3rd table incorporating the Geo table (in addition to tweet and user tables that you already have from HW4 and HW5) and extend your schema accordingly. You do not need to use ALTER TABLE, it is sufficient to just re-make your schema.__

__You will need to generate an ID for the Geo table primary key (you may use any value or reasonable combination of values as long as it is unique) for that table and link it to the Tweet table (foreign key should be in the Tweet). In addition to the primary key column, the geo table should have at least the “type”, “longitude” and “latitude” columns.__


In [182]:
#imports
import urllib.request, time, json, sqlite3

#setup
conn = sqlite3.connect('Tweets_Database_THF1.db') #db connection
c = conn.cursor()
wFD = urllib.request.urlopen('http://rasinsrv07.cstcis.cti.depaul.edu/CSC455/OneDayOfTweets.txt') #get the file

In [149]:
#c.execute('DROP TABLE IF EXISTS User');
#c.execute('DROP TABLE IF EXISTS Tweets');
#c.execute('DROP TABLE IF EXISTS Geo');
wFD.close()
fdErr.close()
c.close()
conn.commit()
conn.close()

In [151]:
#create User Table
create_UserTable = '''CREATE TABLE User (
             ID               INTEGER,
             NAME             TEXT,
             SCREEN_NAME      TEXT,
             DESCRIPTION      TEXT,
             FRIENDS_COUNT    INTEGER,
             CONSTRAINT User_pk  PRIMARY KEY(ID)
          );'''
c.execute('DROP TABLE IF EXISTS User')
c.execute(create_UserTable)

In [155]:
#create Tweets Table
create_TweetsTable = '''CREATE TABLE Tweets (
             ID                      INTEGER,
             Created_At              DATE,
             Text                    TEXT,
             Source                  TEXT,
             In_Reply_to_User_ID     INTEGER,
             In_Reply_to_Screen_Name TEXT,
             In_Reply_to_Status_ID   INTEGER,
             Retweet_Count           INTEGER,
             Contributors            TEXT,
             User_ID                 INTEGER,
             Geo_ID                  Text,
             
             CONSTRAINT Tweet_pk  PRIMARY KEY(ID),
             CONSTRAINT tweet_fk1 FOREIGN KEY (User_ID) REFERENCES User(ID),
             CONSTRAINT tweet_fk2 FOREIGN KEY (Geo_ID) REFERENCES Geo(ID)
             
          );'''
c.execute('DROP TABLE IF EXISTS Tweets')
c.execute(create_TweetsTable)

In [154]:
#create Geo Table
create_GeoTable = '''CREATE TABLE Geo (
             ID            Text,
             Type          Text,
             Latitude      INTEGER,
             Longitude     INTEGER,
             CONSTRAINT Geo_pk PRIMARY KEY(ID)
          );'''
c.execute('DROP TABLE IF EXISTS Geo')
c.execute(create_GeoTable)

__b. Use python to download from the web and save to a local text file (not into database yet, just to text file) at least 1,000,000 lines worth of tweets. Test your code with fewer rows first and only time it when you know it works. Report how long did it take.__

__NOTE: Do not call read() or readlines() without any parameters at any point. That command will attempt to read the entire file which is too much data.__

In [13]:
#open files
start = time.time()
db_file = open('THF_db.txt', 'w') #
db_err_file = open('THF_db_errors.txt', 'w')

for i in range(1000000): #for lines 1 through 1,000,0000
    line = wFD.readline()
    try:
        db_file.write(line.decode()) #write to the database txt file
    except ValueError:
        db_err_file.write(line.decode() + '\n') #catch errors if they come up

#close files
db_file.close()
db_err_file.close()
end   = time.time()

print("Part b file writing took ", (end-start), ' seconds.')

Part b file writing took  1662.4021990299225  seconds.


__c. Repeat what you did in part-b, but instead of saving tweets to the file, populate the 3-table schema that you created in SQLite. Be sure to execute commit and verify that the data has been successfully loaded (report loaded row counts for each of the 3 tables).__

In [124]:
start = time.time()
fdErr = open('THF_error.txt', 'w', errors = 'replace')
tweetBatch = []
userBatch = []
geoBatch =  []
loadCounter = 0

# There is a total of 1,000,000 tweets, but we will do a for-loop here
for i in range(1000000):
    
    line = wFD.readline()
    
    try:  
        tweetDict = json.loads(line) # This is the dictionary for tweet info
        loadCounter = loadCounter + 1

        #------------------------------------
        #Tweet Table
        newRowTweet = [] # hold individual values of to-be-inserted row
        tweetKeys = ['id_str','created_at','text','source','in_reply_to_user_id', 
                     'in_reply_to_screen_name', 'in_reply_to_status_id', 'retweet_count', 'contributors']
        
        for key in tweetKeys: # For each dictionary key we want
            if tweetDict[key] == 'null' or tweetDict[key] == '':
                newRowTweet.append(None) #null
            else:
                newRowTweet.append(tweetDict[key]) # use value as-is
        
        #Adds in user_id
        userDict = tweetDict['user'] # This the the dictionary for user information
        newRowTweet.append(userDict['id']) # User id/ foreign key
        
        #Adds in geo_id
        geoDict = tweetDict['geo']
        if tweetDict['geo']:
            newRowTweet.append(str(tweetDict['geo']['coordinates'])) #geo_id is the latitude/longitude as a string
        else:
            newRowTweet.append(None) # Geo info is missing
        
        #batching
        if loadCounter < 50: # Batching 1 at a time
            tweetBatch.append(newRowTweet)
        else:
            c.executemany ('INSERT OR IGNORE INTO Tweets VALUES(?,?,?,?,?,?,?,?,?,?,?)', tweetBatch)
            tweetBatch = [] # Reset the list of batched tweets
        
        #------------------------------------
        #User Table
        newRowUser = [] # hold individual values of to-be-inserted row for user table
        userKeys = ['id', 'name', 'screen_name', 'description', 'friends_count']

        for key in userKeys: # For each dictionary key we want
            if userDict[key] == 'null' or userDict[key] == '':
                newRowUser.append(None)   # proper NULL
            else:
                newRowUser.append(userDict[key]) # use value as-is
        
        #batching
        if loadCounter < 50: # Batching 1 at a time
            userBatch.append(newRowUser)
        else:
            c.executemany ('INSERT OR IGNORE INTO User VALUES(?,?,?,?,?)', userBatch)
            loadCounter = 0
            userBatch = [] # Reset the list of batched users

        #------------------------------------
        #Geo Table
        newRowGeo = [] # hold individual values of to-be-inserted row for geo table
        geoKeys = ['id','type','latitude', 'longitude']
        
        if tweetDict['geo'] == 'null' or tweetDict['geo'] == '' or tweetDict['geo'] is None:
            #do nothing
            continue
        else:
            #id
            newRowGeo.append(str(tweetDict['geo']['coordinates']))
            #type
            newRowGeo.append(tweetDict['geo']['type'])
            #latitude
            newRowGeo.append(tweetDict['geo']['coordinates'][0])
            #longitude
            newRowGeo.append(tweetDict['geo']['coordinates'][1])
                

        if loadCounter < 50: # Batching 1 at a time
            geoBatch.append(newRowGeo)
        else:
            c.executemany ('INSERT OR IGNORE INTO User VALUES(?,?,?,?)', geoBatch)
            loadCounter = 0
            geoBatch = [] # Reset the list of batched geos

    except ValueError:  # Handle the error of JSON parsing
        fdErr.write(line.decode() + '\n')        

# Final batch (the remaining less-than-50 rows to be loaded)
c.executemany ('INSERT OR IGNORE INTO Tweets VALUES(?,?,?,?,?,?,?,?,?,?,?)', tweetBatch)
c.executemany ('INSERT OR IGNORE INTO User VALUES(?,?,?,?,?)', userBatch)
c.executemany ('INSERT OR IGNORE INTO Geo VALUES(?,?,?,?)', geoBatch)

print ("Loaded ", c.execute('SELECT COUNT(*) FROM Tweets').fetchall()[0], " Tweet rows")
print ("Loaded ", c.execute('SELECT COUNT(*) FROM User').fetchall()[0], " User rows")
print ("Loaded ", c.execute('SELECT COUNT(*) FROM Geo').fetchall()[0], " Geo rows")

wFD.close()
fdErr.close()
c.close()
conn.commit()
conn.close()

end   = time.time()
print("Part c file writing took ", (end-start), ' seconds.')

Loaded  (732324,)  Tweet rows
Loaded  (641680,)  User rows
Loaded  (18084,)  Geo rows
Part c file writing took  479.5136480331421  seconds.


In [132]:
c.execute('SELECT * FROM Geo LIMIT 2').fetchall()

[('[14.670275, 121.043955]', 'Point', 14.670275, 121.043955),
 ('[-7.351872, 110.213471]', 'Point', -7.351872, 110.213471)]

In [133]:
c.execute('SELECT * FROM Tweets LIMIT 2').fetchall()

[(471803285733519360,
  'Thu May 29 00:00:43 +0000 2014',
  'Happy world burger day everyone!',
  '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
  None,
  None,
  None,
  0,
  None,
  462658467,
  None),
 (471803285733531648,
  'Thu May 29 00:00:43 +0000 2014',
  "party at Taryn's come RT RT RT http://t.co/4vTC5VGV9X",
  '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
  None,
  None,
  None,
  0,
  None,
  577731711,
  None)]

In [134]:
c.execute('SELECT * FROM User LIMIT 2').fetchall()

[(3065,
  'Flávia Del Rio',
  'flaviadurante',
  'Jornalista, DJ e music freak. Paulistana criada em #SANTOSmelhoremtudo',
  3988),
 (3300, 'Jonathan Wight', 'schwa', 'No.', 532)]

__How long did this step take?__

It took:

__d. Use your locally saved tweet file (created in part-b) to repeat the database population step from part-c. That is, load 1,000,000 tweets into the 3-table database using your saved file with tweets (do not use the URL to read twitter data).__

In [147]:
start = time.time()
#open the database text file
f = open("THF_db.txt", 'r', encoding='utf-8')

fdErr = open('THF_error.txt', 'w', errors = 'replace')
tweetBatch = []
userBatch = []
geoBatch =  []
loadCounter = 0

# There is a total of 1,000,000 tweets, but we will do a for-loop here
for i in range(1000000):
    
    line = f.readline()
    
    try:  
        tweetDict = json.loads(line) # This is the dictionary for tweet info
        loadCounter = loadCounter + 1

        #------------------------------------
        #Tweet Table
        newRowTweet = [] # hold individual values of to-be-inserted row
        tweetKeys = ['id_str','created_at','text','source','in_reply_to_user_id', 
                     'in_reply_to_screen_name', 'in_reply_to_status_id', 'retweet_count', 'contributors']
        
        for key in tweetKeys: # For each dictionary key we want
            if tweetDict[key] == 'null' or tweetDict[key] == '':
                newRowTweet.append(None) #null
            else:
                newRowTweet.append(tweetDict[key]) # use value as-is
        
        #Adds in user_id
        userDict = tweetDict['user'] # This the the dictionary for user information
        newRowTweet.append(userDict['id']) # User id/ foreign key
        
        #Adds in geo_id
        geoDict = tweetDict['geo']
        if tweetDict['geo']:
            newRowTweet.append(str(tweetDict['geo']['coordinates'])) #geo_id is the latitude/longitude as a string
        else:
            newRowTweet.append(None) # Geo info is missing
        
        #batching
        if loadCounter < 50: # Batching 1 at a time
            tweetBatch.append(newRowTweet)
        else:
            c.executemany ('INSERT OR IGNORE INTO Tweets VALUES(?,?,?,?,?,?,?,?,?,?,?)', tweetBatch)
            tweetBatch = [] # Reset the list of batched tweets
        
        #------------------------------------
        #User Table
        newRowUser = [] # hold individual values of to-be-inserted row for user table
        userKeys = ['id', 'name', 'screen_name', 'description', 'friends_count']

        for key in userKeys: # For each dictionary key we want
            if userDict[key] == 'null' or userDict[key] == '':
                newRowUser.append(None)   # proper NULL
            else:
                newRowUser.append(userDict[key]) # use value as-is
        
        #batching
        if loadCounter < 50: # Batching 1 at a time
            userBatch.append(newRowUser)
        else:
            c.executemany ('INSERT OR IGNORE INTO User VALUES(?,?,?,?,?)', userBatch)
            loadCounter = 0
            userBatch = [] # Reset the list of batched users

        #------------------------------------
        #Geo Table
        newRowGeo = [] # hold individual values of to-be-inserted row for geo table
        geoKeys = ['id','type','latitude', 'longitude']
        
        if tweetDict['geo'] == 'null' or tweetDict['geo'] == '' or tweetDict['geo'] is None:
            #do nothing
            continue
        else:
            #id
            newRowGeo.append(str(tweetDict['geo']['coordinates']))
            #type
            newRowGeo.append(tweetDict['geo']['type'])
            #latitude
            newRowGeo.append(tweetDict['geo']['coordinates'][0])
            #longitude
            newRowGeo.append(tweetDict['geo']['coordinates'][1])
                

        if loadCounter < 50: # Batching 1 at a time
            geoBatch.append(newRowGeo)
        else:
            c.executemany ('INSERT OR IGNORE INTO User VALUES(?,?,?,?)', geoBatch)
            loadCounter = 0
            geoBatch = [] # Reset the list of batched geos

    except ValueError:  # Handle the error of JSON parsing
        fdErr.write(line.decode() + '\n')        

# Final batch (the remaining less-than-50 rows to be loaded)
c.executemany ('INSERT OR IGNORE INTO Tweets VALUES(?,?,?,?,?,?,?,?,?,?,?)', tweetBatch)
c.executemany ('INSERT OR IGNORE INTO User VALUES(?,?,?,?,?)', userBatch)
c.executemany ('INSERT OR IGNORE INTO Geo VALUES(?,?,?,?)', geoBatch)

print ("Loaded ", c.execute('SELECT COUNT(*) FROM Tweets').fetchall()[0], " Tweet rows")
print ("Loaded ", c.execute('SELECT COUNT(*) FROM User').fetchall()[0], " User rows")
print ("Loaded ", c.execute('SELECT COUNT(*) FROM Geo').fetchall()[0], " Geo rows")

f.close()

wFD.close()
fdErr.close()
c.close()
conn.commit()
conn.close()
end   = time.time()
print("Part d file writing took ", (end-start), ' seconds.')

Loaded  (979569,)  Tweet rows
Loaded  (840278,)  User rows
Loaded  (23993,)  Geo rows
Part d file writing took  585.7131180763245  seconds.


__How does the runtime compare with part-c?__

Compared to part-c it took:

__e. Re-run the previous step with a batching size of 1000 (i.e. by inserting 1000 rows at a time with executemany).__

In [157]:
start = time.time()
#open the database text file
f = open("THF_db.txt", 'r', encoding='utf-8')

fdErr = open('THF_error.txt', 'w', errors = 'replace')
tweetBatch = []
userBatch = []
geoBatch =  []
loadCounter = 0

# There is a total of 1,000,000 tweets, but we will do a for-loop here
for i in range(1000000):
    
    line = f.readline()
    
    try:  
        tweetDict = json.loads(line) # This is the dictionary for tweet info
        loadCounter = loadCounter + 1

        #------------------------------------
        #Tweet Table
        newRowTweet = [] # hold individual values of to-be-inserted row
        tweetKeys = ['id_str','created_at','text','source','in_reply_to_user_id', 
                     'in_reply_to_screen_name', 'in_reply_to_status_id', 'retweet_count', 'contributors']
        
        for key in tweetKeys: # For each dictionary key we want
            if tweetDict[key] == 'null' or tweetDict[key] == '':
                newRowTweet.append(None) #null
            else:
                newRowTweet.append(tweetDict[key]) # use value as-is
        
        #Adds in user_id
        userDict = tweetDict['user'] # This the the dictionary for user information
        newRowTweet.append(userDict['id']) # User id/ foreign key
        
        #Adds in geo_id
        geoDict = tweetDict['geo']
        if tweetDict['geo']:
            newRowTweet.append(str(tweetDict['geo']['coordinates'])) #geo_id is the latitude/longitude as a string
        else:
            newRowTweet.append(None) # Geo info is missing
        
        #batching
        if loadCounter < 1000: # Batching 1 at a time
            tweetBatch.append(newRowTweet)
        else:
            c.executemany ('INSERT OR IGNORE INTO Tweets VALUES(?,?,?,?,?,?,?,?,?,?,?)', tweetBatch)
            tweetBatch = [] # Reset the list of batched tweets
        
        #------------------------------------
        #User Table
        newRowUser = [] # hold individual values of to-be-inserted row for user table
        userKeys = ['id', 'name', 'screen_name', 'description', 'friends_count']

        for key in userKeys: # For each dictionary key we want
            if userDict[key] == 'null' or userDict[key] == '':
                newRowUser.append(None)   # proper NULL
            else:
                newRowUser.append(userDict[key]) # use value as-is
        
        #batching
        if loadCounter < 1000: # Batching 1 at a time
            userBatch.append(newRowUser)
        else:
            c.executemany ('INSERT OR IGNORE INTO User VALUES(?,?,?,?,?)', userBatch)
            loadCounter = 0
            userBatch = [] # Reset the list of batched users

        #------------------------------------
        #Geo Table
        newRowGeo = [] # hold individual values of to-be-inserted row for geo table
        geoKeys = ['id','type','latitude', 'longitude']
        
        if tweetDict['geo'] == 'null' or tweetDict['geo'] == '' or tweetDict['geo'] is None:
            #do nothing
            continue
        else:
            #id
            newRowGeo.append(str(tweetDict['geo']['coordinates']))
            #type
            newRowGeo.append(tweetDict['geo']['type'])
            #latitude
            newRowGeo.append(tweetDict['geo']['coordinates'][0])
            #longitude
            newRowGeo.append(tweetDict['geo']['coordinates'][1])
                

        if loadCounter < 1000: # Batching 1 at a time
            geoBatch.append(newRowGeo)
        else:
            c.executemany ('INSERT OR IGNORE INTO User VALUES(?,?,?,?)', geoBatch)
            loadCounter = 0
            geoBatch = [] # Reset the list of batched geos

    except ValueError:  # Handle the error of JSON parsing
        fdErr.write(line.decode() + '\n')        

# Final batch (the remaining less-than-50 rows to be loaded)
c.executemany ('INSERT OR IGNORE INTO Tweets VALUES(?,?,?,?,?,?,?,?,?,?,?)', tweetBatch)
c.executemany ('INSERT OR IGNORE INTO User VALUES(?,?,?,?,?)', userBatch)
c.executemany ('INSERT OR IGNORE INTO Geo VALUES(?,?,?,?)', geoBatch)

print ("Loaded ", c.execute('SELECT COUNT(*) FROM Tweets').fetchall()[0], " Tweet rows")
print ("Loaded ", c.execute('SELECT COUNT(*) FROM User').fetchall()[0], " User rows")
print ("Loaded ", c.execute('SELECT COUNT(*) FROM Geo').fetchall()[0], " Geo rows")

f.close()

wFD.close()
fdErr.close()
c.close()
conn.commit()
conn.close()
end   = time.time()
print("Part e file writing took ", (end-start), ' seconds.')

Loaded  (998554,)  Tweet rows
Loaded  (854707,)  User rows
Loaded  (23993,)  Geo rows
Part d file writing took  348.1966998577118  seconds.


__How does the runtime compare when batching is used?__

The runtime with batching:

### Problem 2
__a. Write and execute SQL queries to do the following. Don’t forget to report the running times in each part and the code you used.__

__i. Find tweets where tweet id_str contains “55” or “88” anywhere in the column__

In [166]:
start = time.time()
c.execute('SELECT * FROM Tweets WHERE id LIKE "%55%" or id LIKE "%88%" ').fetchall()
end = time.time()
print("Part i query took ", (end-start), ' seconds.')

Part e file writing took  5.004155158996582  seconds.


In [167]:
c.execute('SELECT * FROM Tweets WHERE id LIKE "%55%" or id LIKE "%88%" ').fetchall()

[(471803285738106880,
  'Thu May 29 00:00:43 +0000 2014',
  'Mucho la Plop esto, la Plop aquello, pero de los viernes es la fiesta con la gente más linda. \nEn las otras vienen directo de la frontera.',
  'web',
  None,
  None,
  None,
  0,
  None,
  38950479,
  None),
 (471803285742288896,
  'Thu May 29 00:00:43 +0000 2014',
  'RT @beyles66: cant wait for next year with @Harrison_BHS and @ThePurbalite',
  '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
  None,
  None,
  None,
  0,
  None,
  329489005,
  None),
 (471803285746495488,
  'Thu May 29 00:00:43 +0000 2014',
  '“@CrispyCairo: your ex when yall together | when yall break up. 😂😩💯 http://t.co/YU72KlDHKk” 😂😂😂😭',
  '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
  347500716,
  'CrispyCairo',
  471773262171476000,
  0,
  None,
  468079049,
  None),
 (471803285754490881,
  'Thu May 29 00:00:43 +0000 2014',
  '@ChoiiSul ah kamu;(',
  '<a href="http://twitte

__ii. Find how many unique values are there in the “in_reply_to_user_id” column__

In [169]:
start = time.time()
c.execute('SELECT COUNT(DISTINCT in_reply_to_user_id) AS num_replies FROM Tweets').fetchall()
end = time.time()
print("Part ii query took ", (end-start), ' seconds.')

Part ii file writing took  1.919631004333496  seconds.


In [171]:
c.execute('SELECT COUNT(DISTINCT in_reply_to_user_id) AS num_replies FROM Tweets').fetchall()

[(180243,)]

__iii. Find the tweet(s) with the shortest, longest and average length text message.__

In [188]:
start = time.time()
c.execute('SELECT MIN(LENGTH(Text)) AS shortest, MAX(LENGTH(Text)) AS longest, AVG(LENGTH(Text)) \
            AS average FROM Tweets').fetchall()
end = time.time()
print("Part iii query took ", (end-start), ' seconds.')

Part iii file writing took  0.9016211032867432  seconds.


In [189]:
start = time.time()
c.execute('SELECT * FROM TWEETS WHERE LENGTH(Text) IN (1,434,68.83193998521863)').fetchall()
end = time.time()
print("Part iii query took ", (end-start), ' seconds.')

Part iii file writing took  0.5746147632598877  seconds.


__iv. Find the average longitude and latitude value for each user name.__

In [191]:
start = time.time()
c.execute('SELECT screen_name, AVG(latitude), AVG(longitude) FROM User \
        JOIN Tweets ON User.ID=Tweets.user_id \
        JOIN Geo ON Tweets.geo_id=Geo.ID \
        GROUP BY screen_name').fetchall()
end = time.time()
print("Part iv query took ", (end-start), ' seconds.')

Part iv query took  6.078100919723511  seconds.


In [192]:
c.execute('SELECT screen_name, AVG(latitude), AVG(longitude) FROM User \
        JOIN Tweets ON User.ID=Tweets.user_id \
        JOIN Geo ON Tweets.geo_id=Geo.ID \
        GROUP BY screen_name').fetchall()

[('0035112', 35.035168, 135.730054),
 ('007LouiseOB', 40.729836, -73.997618),
 ('007Malena', -38.046585, -57.582788),
 ('0112_ayu', 35.010069, 135.684231),
 ('011Freygo', 30.324492, -81.656281),
 ('02_antoo', -53.797996, -67.726057),
 ('02yuka081', 35.610051, 139.569266),
 ('0330Ke', 48.580178, 7.732287),
 ('03Avivalau', -28.46225, -65.73199),
 ('03_Desember', -7.309173, 112.582887),
 ('03bda', 52.252056, 1.208712),
 ('03maaaam', 36.566099, 139.853305),
 ('0500515495', 28.344589, 36.596308),
 ('0502228153', 30.99845, 41.061585),
 ('0509Shion', 34.777574, 134.801135),
 ('0522_fam', 35.966739, 139.687625),
 ('0533598755gamal', 24.321882, 45.742213),
 ('0602Gmai', 42.833353, 141.650445),
 ('0603ponnnta', 35.446969, 139.956389),
 ('0618y_y', 43.058428, 141.326849),
 ('0630Senya', 35.714049, 139.705026),
 ('06aimeeg', -33.995688, 25.655968),
 ('0770Abdulwahab', 38.792454, -77.201505),
 ('07Know', 34.70656, 135.492951),
 ('07_azusa', 34.832544, 135.706497),
 ('08Sandes', -32.828335, -61.4077

__v. Find how many known/unknown locations there were in total (e.g., 50,000 known, 950,000 unknown,  5% locations are available)__ 

In [224]:
c.execute('SELECT (COUNT(*)-COUNT(CASE WHEN `geo_id` IS NULL THEN 1 END)) \
                    ,COUNT(CASE WHEN `geo_id` IS NULL THEN 1 END), \
                    ROUND((COUNT(*)-COUNT(CASE WHEN `geo_id` IS NULL THEN 1 END)) * 100.0 /     \
                    COUNT(CASE WHEN `geo_id` IS NULL THEN 1 END), 1)FROM Tweets').fetchall()

[(24291, 974263, 2.5)]

In [225]:
start = time.time()
c.execute('SELECT (COUNT(*)-COUNT(CASE WHEN `geo_id` IS NULL THEN 1 END)) \
                    ,COUNT(CASE WHEN `geo_id` IS NULL THEN 1 END), \
                    ROUND((COUNT(*)-COUNT(CASE WHEN `geo_id` IS NULL THEN 1 END)) * 100.0 /     \
                    COUNT(CASE WHEN `geo_id` IS NULL THEN 1 END), 1)FROM Tweets').fetchall()
end = time.time()
print("Part v query took ", (end-start), ' seconds.')

Part v query took  0.8259048461914062  seconds.


__vi. Re-execute the query in part iv) 10 times and 100 times and measure the total runtime (just re-run the same exact query multiple times using a for-loop). Does the runtime scale linearly? (i.e., does it take 10X and 100X as much time?)__

In [194]:
start = time.time()

for i in range(10):
    c.execute('SELECT screen_name, AVG(latitude), AVG(longitude) FROM User \
        JOIN Tweets ON User.ID=Tweets.user_id \
        JOIN Geo ON Tweets.geo_id=Geo.ID \
        GROUP BY screen_name').fetchall()
end = time.time()
print("Part iv 10x query took ", (end-start), ' seconds.')

Part iv 10x query took  62.96931505203247  seconds.


In [195]:
start = time.time()

for i in range(100):
    c.execute('SELECT screen_name, AVG(latitude), AVG(longitude) FROM User \
        JOIN Tweets ON User.ID=Tweets.user_id \
        JOIN Geo ON Tweets.geo_id=Geo.ID \
        GROUP BY screen_name').fetchall()
end = time.time()
print("Part iv 100x query took ", (end-start), ' seconds.')

Part iv 100x query took  605.360847234726  seconds.


__b. Write python code that is going to read the locally saved tweet data file from 1-b and perform the equivalent computation for parts 2-i and 2-ii only. How does the runtime compare to the SQL queries?__

In [243]:
#i - c.execute('SELECT * FROM Tweets WHERE id LIKE "%55%" or id LIKE "%88%" ').fetchall()
import pandas as pd

start = time.time()
f = open("THF_db.txt", 'r', encoding='utf-8')

data = []
labels = ['id_str','in_reply_to_user_id']
error_tally =0
# Loop through the 1,000,000 tweets in the text file
for i in range(1000000):
    
    line = f.readline()
    
    try:  
        tweetDict = json.loads(line) # This is the dictionary for tweet info
        data.append((tweetDict["id_str"], tweetDict["in_reply_to_user_id"]))
    
    except:  #catch any error
        error_tally+=1

df=pd.DataFrame.from_records(data,columns=labels)

f.close() 
df_end = df[df['id_str'].astype(str).str.contains('55|88')]

end = time.time()
print("Part 2b-i loop took ", (end-start), ' seconds.')

Part 2b-i loop took  103.7078309059143  seconds.


In [244]:
df_end.head(10)

,id_str,in_reply_to_user_id
1,471803285738106880,NaN
7,471803285742288896,NaN
15,471803285762883585,NaN
30,471803285771255808,NaN
32,471803285762887680,NaN
34,471803285754490881,1.943298e+09
41,471803285746495488,3.475007e+08
62,471803289953402880,1.363348e+09
67,471803289927831553,NaN
72,471803289948811264,1.582026e+08


In [258]:
#ii - c.execute('SELECT COUNT(DISTINCT in_reply_to_user_id) AS num_replies FROM Tweets').fetchall()
import pandas as pd

start = time.time()
f = open("THF_db.txt", 'r', encoding='utf-8')

data = []
labels = ['id_str','in_reply_to_user_id']
error_tally =0
# Loop through the 1,000,000 tweets in the text file
for i in range(1000000):
    
    line = f.readline()
    
    try:  
        tweetDict = json.loads(line) # This is the dictionary for tweet info
        data.append((tweetDict["id_str"], tweetDict["in_reply_to_user_id"]))
    
    except:  #catch any error
        error_tally+=1

df=pd.DataFrame.from_records(data,columns=labels)
f.close() 

df_end = df['in_reply_to_user_id'].value_counts(ascending=False)

end = time.time()
print("Part 2b-ii loop took ", (end-start), ' seconds.')

Part 2b-ii loop took  100.09605598449707  seconds.


In [259]:
df_end.head(10)

3.562471e+06    1057
2.726009e+07     621
1.051195e+08     229
3.130879e+08     206
2.099942e+08     196
1.396698e+09     163
4.364136e+07     158
1.715134e+07     137
1.815617e+08     128
1.807244e+09     122
Name: in_reply_to_user_id, dtype: int64

### Problem 3

__a. Export the contents of the User table from a SQLite table into a sequence of INSERT statements within a file. This is very similar to what you already did in Assignment 4. However, you have to add a unique ID column which has to be a string (you cannot use numbers). Hint: you can replace digits with letters, e.g., chr(ord('a')+1) gives you a 'b' and chr(ord('a')+2) returns a 'c'__
 

In [262]:
#import sqlite3

def generateInsertStatements(tblName):
    conn = sqlite3.connect('Tweets_Database_THF1.db') # Using HW3 SQLite DB (preloaded)
    c = conn.cursor()

    # Open file for export
    fd = open(tblName+'.txt', 'w')

    tblRows = c.execute('SELECT * FROM %s' % tblName)

    for row in tblRows:
        fd.write("INSERT INTO %s VALUES %s;\n" % (tblName, str(row)))
        
    fd.close()
    c.close()
    conn.close()

start = time.time()
generateInsertStatements('User')
end = time.time()
print("Part 3a loop took ", (end-start), ' seconds.')

Part 2b-ii loop took  314.5617868900299  seconds.


Part 2b-ii loop took  314.5617868900299  seconds.

__b. Create the same collection of INSERT for the User table by reading data from the local tweet file that you have saved earlier.__

In [269]:
def generateInsertStatements_b(tblName):
    #open the database text file
    f = open("THF_db.txt", 'r', encoding='utf-8')
    #open the file to write to
    fd = open(tblName+'.txt', 'w')
    err=0
    for i in range(1000000):
        line = f.readline()
        try:  
            tweetDict = json.loads(line) # This is the dictionary for tweet info
            userDict = tweetDict['user']
            
           #User Table
            newRowUser = [] # hold individual values of to-be-inserted row for user table
            userKeys = ['id', 'name', 'screen_name', 'description', 'friends_count']

            for key in userKeys: # For each dictionary key we want
                if userDict[key] == 'null' or userDict[key] == '':
                    newRowUser.append(None)   # proper NULL
                else:
                    newRowUser.append(userDict[key]) # use value as-is
            fd.write("INSERT INTO %s VALUES %s;\n" % (tblName, str((newRowUser))))
        except:
            err+=1
    f.close()
    fd.close()
start = time.time()
generateInsertStatements_b('User')
end = time.time()
print("Part 3b loop took ", (end-start), ' seconds.')

Part 3b loop took  134.9316029548645  seconds.


__How do these compare in runtime? Which method was faster?__

Comparing the runtime:

### Problem 4

__4. Export all three tables (Tweet, User and Geo tables) from the database into a |-separated text file (each value in a row should be separated by |). You do not generate SQL INSERT statements, just raw |-separated text data.__

In [ ]:
#import sqlite3 
#import pandas as pd
conn = sqlite3.connect('Tweets_Database_THF1.db')
c = conn.cursor()


df_tweets_read = pd.read_sql_query("SELECT * FROM Tweets;", conn) #tweets
df_user_read = pd.read_sql_query("SELECT * FROM User;", conn) #user
df_geo_read = pd.read_sql_query("SELECT * FROM Geo;", conn) #geo

df_tweets_write = df_tweets_read.to_csv("tweets_table.txt", sep ='|') #tweets
df_user_write = df_user_read.to_csv("user_table.txt", sep ='|') #user
df_geo_write = df_geo_read.to_csv("geo_table.txt", sep ='|') #geo



c.close()
conn.commit()
conn.close()

__a. For the Geo table, add a new column with relative distance from a fixed point which is the location of CDM (41.878668, -87.625555). You can simply treat it as a point-to-point Euclidean distance (although bonus points for finding a real distance in miles) and round the longitude and latitude columns to a maximum of 4 digits after the decimal.__
 

In [284]:
import sqlite3 
import pandas as pd
import numpy as np
conn = sqlite3.connect('Tweets_Database_THF1.db')
c = conn.cursor()

df_geo_read = pd.read_sql_query("SELECT * FROM Geo;", conn) #geo

df_geo_read['Latitude'] = df_geo_read.Latitude.round(4)
df_geo_read['Longitude'] = df_geo_read.Longitude.round(4)
df_geo_read['distance'] = (df_geo_read.Latitude.sub(41.878668).pow(2).add(df_geo_read.Longitude.sub(-87.625555).pow(2))).pow(.5).round(4)

df_geo_write = df_geo_read.to_csv("geo_table.txt", sep ='|') #geo

c.close()
conn.commit()
conn.close()
df_geo_read.head(10)

,ID,Type,Latitude,Longitude,distance
0,"[14.670275, 121.043955]",Point,14.6703,121.0440,210.4359
1,"[-7.351872, 110.213471]",Point,-7.3519,110.2135,203.8724
2,"[47.8487, -122.222]",Point,47.8487,-122.2220,35.1078
3,"[38.767654, -77.159617]",Point,38.7677,-77.1596,10.9185
4,"[-6.149429, 106.728999]",Point,-6.1494,106.7290,200.2009
5,"[52.536283, -1.505078]",Point,52.5363,-1.5051,86.7774
6,"[26.198516, 127.698844]",Point,26.1985,127.6988,215.8945
7,"[36.109317, -115.170286]",Point,36.1093,-115.1703,28.1425
8,"[-23.953996, -46.288466]",Point,-23.9540,-46.2885,77.7348
9,"[-29.188215, -51.216072]",Point,-29.1882,-51.2161,79.8508


__b. For the Tweet table, add two new columns from the User table (“name” and “screen_name”) in addition to existing columns.__
 

In [295]:
import sqlite3 
import pandas as pd
conn = sqlite3.connect('Tweets_Database_THF1.db')
c = conn.cursor()

df_tweets_read = pd.read_sql_query("SELECT * FROM Tweets;", conn) #Tweets
df_user_read = pd.read_sql_query("SELECT * FROM User;", conn) #User

new_df = pd.merge(df_tweets_read, df_user_read, how='left', left_on='User_ID',right_on='ID')
new_df = new_df.drop(['DESCRIPTION','FRIENDS_COUNT','ID_y'], axis=1)

df_tweets_write = new_df.to_csv("tweet_table.txt", sep ='|') #Tweets written

c.close()
conn.commit()
conn.close()

In [296]:
new_df.head(10)

,ID_x,Created_At,Text,Source,In_Reply_to_User_ID,In_Reply_to_Screen_Name,In_Reply_to_Status_ID,Retweet_Count,Contributors,User_ID,Geo_ID,NAME,SCREEN_NAME
0,471803285733519360,Thu May 29 00:00:43 +0000 2014,Happy world burger day everyone!,"<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,0,None,462658467,None,Dyl,Big_Dyl97
1,471803285733531648,Thu May 29 00:00:43 +0000 2014,party at Taryn's come RT RT RT http://t.co/4vT...,"<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,0,None,577731711,None,t a r y n,TarynThomas2
2,471803285733531649,Thu May 29 00:00:43 +0000 2014,RT @RealTonyRocha: #twitter Twitter CEO Says I...,"<a href=""http://tweetadder.com"" rel=""nofollow""...",NaN,None,NaN,0,None,2419839284,None,Melissah Macwhirter,veveciwiluvi
3,471803285733904386,Thu May 29 00:00:43 +0000 2014,Photo: http://t.co/wy3XTCGw6u,"<a href=""http://www.tumblr.com/"" rel=""nofollow...",NaN,None,NaN,0,None,1100829277,None,Claudia ₪ ø lll ·o.,Kawaiiotakugurl
4,471803285733916672,Thu May 29 00:00:43 +0000 2014,RT @gagationer: Just ripped up my One Directio...,"<a href=""http://twitter.com/download/iphone"" r...",NaN,None,NaN,0,None,339805930,None,PLEASE CALUM ILYSM,HumpingHaroldS
5,471803285737717760,Thu May 29 00:00:43 +0000 2014,@null 79,"<a href=""http://twittbot.net/"" rel=""nofollow"">...",3.562471e+06,None,NaN,0,None,2382808592,None,クルーク,syuusai_klug
6,471803285737717761,Thu May 29 00:00:43 +0000 2014,"""@hanyacintadia: Aku selalu tersenyum disaat a...","<a href=""http://blackberry.com/twitter"" rel=""n...",NaN,None,NaN,0,None,598244144,None,Anarciz_Autis,Friskasriputri
7,471803285737721856,Thu May 29 00:00:43 +0000 2014,9:00です☆今日のお天気何かな？居抜き店舗探しはhttp://t.co/IzSeSUIQh2,"<a href=""http://twittbot.net/"" rel=""nofollow"">...",NaN,None,NaN,0,None,317649466,None,店舗そのままオークション広島市,HiroshimaTenpo
8,471803285738094592,Thu May 29 00:00:43 +0000 2014,“@J_Mf_Gandee26: This picture 😍💋👯🍻👌😩 @Pleasure...,"<a href=""http://twitter.com/download/iphone"" r...",1.083805e+09,J_Mf_Gandee26,4.718031e+17,0,None,561545606,None,baddgirl⭐️,Damn_CeBadd
9,471803285738106880,Thu May 29 00:00:43 +0000 2014,"Mucho la Plop esto, la Plop aquello, pero de l...",web,NaN,None,NaN,0,None,38950479,None,Yarer Sofier,firekites


__c. For the User table file add a column that specifies how many tweets by that user are currently in the database. That is, your output file should contain all of the columns from the User table, plus the new column with tweet count. You do not need to modify the User table, just create the output text file. What is the name of the user with most tweets?__

In [302]:
#x = new_df['User_ID'].value_counts()
#x.columns = ['User_ID','tweets_count']
#x.head()
z = new_df['User_ID']

2374595534    61
1958482998    44
66652699      37
1374693672    22
770980550     22
Name: User_ID, dtype: int64

In [322]:
import sqlite3 
import pandas as pd
conn = sqlite3.connect('Tweets_Database_THF1.db')
c = conn.cursor()

#df_user_read = pd.read_sql_query("SELECT * FROM User;", conn) #User
#df_tweets_read = pd.read_sql_query("SELECT * FROM Tweets;", conn) #Tweets

new_df['tweets_count'] = new_df.groupby('User_ID')['User_ID'].transform('count') #from last part
join_df = new_df[['User_ID','tweets_count']] #dataframe with only id and tweetcount

newer_df = pd.merge(df_user_read, join_df, left_on='ID', right_on='User_ID')

clean_df = newer_df.sort_values(by=['tweets_count'], ascending=False).drop_duplicates()

df_user_write = clean_df.to_csv("user_table.txt", sep ='|') #user

c.close()
conn.commit()
conn.close()

In [323]:
clean_df.head(10)

,ID,NAME,SCREEN_NAME,DESCRIPTION,FRIENDS_COUNT,User_ID,tweets_count
878404,2374595534,Alma Arafat,ArafatPidip,Second Account dari @AfatArafaat,70,2374595534,61
768557,1958482998,HEXAREVOLVER,HEXAREVOLVER,Official account HEXAREVOLVER,299,1958482998,44
51595,66652699,Sports Answerbank,SportsAB,None,8,66652699,37
507705,770980550,ゆうきしゃん\n,85318081,NikonのD40X使ってまーす(´･_･`),156,770980550,22
652669,1374693672,あああああああああああああああああああ,colorado_null,ああああああああああああああああああああああああああああああああああああああああああああああ...,46,1374693672,22
670422,1448837792,None,tweetkasegida,None,0,1448837792,21
740161,1717183141,viaあみの,ValaLell,@7__su←肉便器,189,1717183141,20
621839,1247697384,恋ヶ崎,__5i,お前らが恋するたびに俺が生まれる｡\n(( Sub / @SHlTTO @5118 @51o )),7,1247697384,19
914113,2433446852,سـعــوديــة,arbsssex,None,0,2433446852,18
321706,387431298,Mashaad,ThatsMashaad,Contact: Bookmashaad@gmail.com,2930,387431298,18


In [309]:
newer_df.head(10)

NameError: name 'newer_df' is not defined